<a href="https://colab.research.google.com/github/MengOonLee/AccountReceivable/blob/main/Workflow/InvoicePayment/FineTune_Llama2/Forecast_DaysToSettle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash
pip install --no-cache-dir -qU \
    datasets transformers bitsandbytes accelerate peft trl

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
import torch
import transformers

base_model = "meta-llama/Llama-2-7b-chat-hf"

pretrained_model = transformers.AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.bfloat16,
    device_map='auto'
)

tokenizer = transformers.AutoTokenizer.from_pretrained(
    base_model, use_auth_token=True)
tokenizer.pad_token = tokenizer.eos_token

prompt = "What is Swift?"

pipe = transformers.pipeline(
    task="text-generation", 
    model=pretrained_model,
    tokenizer=tokenizer)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:655: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


<s>[INST] What is Swift? [/INST]  Swift is a programming language developed by Apple Inc. for building software on Apple devices such as macOS, iOS, watchOS, and tvOS. It was announced at Apple's Worldwide Developers Conference (WWDC) in 2014 and is designed to replace Apple's previous programming language, Objective-C.

Swift is a modern, high-performance language that is designed to give developers more freedom to create powerful, easy-to-read, and maintainable code. It is built on a number of innovative concepts, including:

1. Memory Safety: Swift is designed to prevent common programming errors like null pointer dereferences, data races, and buffer overflows. It does this through a combination of static and dynamic analysis, as well as a unique concept called "ownership and borrowing".
2. Performance: Swift is designed to be fast and efficient, with a focus on generating efficient machine code. It uses a number of techniques, including just-in-time (JIT) compilation and compile-ti

In [7]:
prompt = "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nGiven the dataset delimited by the triple backticks, forecast number of days client 2621-XCLEH will take for the payment of an invoice dated 2013-07-28 with an amount $92.17 to be settled. Return the response in JSON format, containing four keys: 'customerID', 'InvoiceDate', 'InvoiceAmount' and 'DaysToSettle'. Return only the forecast, not the Python code.\n\n```\ncustomerID InvoiceDate InvoiceAmount DaysToSettle\n2621-XCLEH  2012-01-13        $80.99      61 days\n2621-XCLEH  2012-02-21        $79.51      46 days\n2621-XCLEH  2012-02-22        $69.80      43 days\n2621-XCLEH  2012-03-02        $67.51      57 days\n2621-XCLEH  2012-03-23        $89.05      52 days\n2621-XCLEH  2012-04-16        $74.06      44 days\n2621-XCLEH  2012-06-27        $69.42      60 days\n2621-XCLEH  2012-11-18        $86.39      75 days\n2621-XCLEH  2013-03-01        $58.96      55 days\n2621-XCLEH  2013-03-31        $70.93      55 days\n2621-XCLEH  2013-04-27        $65.76      37 days\n2621-XCLEH  2013-06-18        $37.49      29 days\n2621-XCLEH  2013-06-24        $90.62      35 days\n2621-XCLEH  2013-07-16        $78.08      48 days\n```\n\n### Response:"

pipe = transformers.pipeline(
    task="text-generation", 
    model=pretrained_model,
    tokenizer=tokenizer)
result = pipe(f"<s>[INST]\n{prompt}\n[/INST]")
print(result[0]['generated_text'])

<s>[INST]
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Given the dataset delimited by the triple backticks, forecast number of days client 2621-XCLEH will take for the payment of an invoice dated 2013-07-28 with an amount $92.17 to be settled. Return the response in JSON format, containing four keys: 'customerID', 'InvoiceDate', 'InvoiceAmount' and 'DaysToSettle'. Return only the forecast, not the Python code.

```
customerID InvoiceDate InvoiceAmount DaysToSettle
2621-XCLEH  2012-01-13        $80.99      61 days
2621-XCLEH  2012-02-21        $79.51      46 days
2621-XCLEH  2012-02-22        $69.80      43 days
2621-XCLEH  2012-03-02        $67.51      57 days
2621-XCLEH  2012-03-23        $89.05      52 days
2621-XCLEH  2012-04-16        $74.06      44 days
2621-XCLEH  2012-06-27        $69.42      60 days
2621-XCLEH  2012-11-18        $86.39      75 days
2621-XCLEH  2013-03-01        $58.96      55 days
26

In [ ]:
import datasets

dataset = datasets.load_dataset("mlabonne/guanaco-llama2-1k", split='train')
print(dataset[500])

In [ ]:
import torch
import transformers

base_model = "meta-llama/Llama-2-7b-chat-hf"

quantization_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quantization_config,
    device_map='auto'
)
model.config.use_cache = False

tokenizer = transformers.AutoTokenizer.from_pretrained(
    base_model, use_auth_token=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
import transformers
import peft
import trl

training_args = transformers.TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=2e-4,
    fp16=True,
    max_steps=15,
    warmup_steps=2
)

peft_config = peft.LoraConfig(
    lora_alpha=64, lora_dropout=0.1, r=16,
    bias='none', task_type='CAUSAL_LM'
)

trainer = trl.SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field='text',
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_args,
    packing=False
)

trainer.train()

peft_model = "peft_model"
trainer.model.save_pretrained(peft_model)

del model
del trainer
torch.cuda.empty_cache()

In [ ]:
import torch
import transformers
import peft

pretrained_model = transformers.AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.bfloat16,
    device_map='auto'
)
finetuned_model = peft.PeftModel.from_pretrained(
    pretrained_model, peft_model
)
finetuned_model = finetuned_model.merge_and_unload()
finetuned_dir = "finetuned_Llama2_forecast_DaysToSettle"
finetuned_model.save_pretrained(finetuned_dir)

tokenizer = transformers.AutoTokenizer.from_pretrained(
    base_model, use_auth_token=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.save_pretrained(finetuned_dir)

In [ ]:
import transformers

prompt = "What is a large language model?"
pipe = transformers.pipeline(
    task="text-generation", 
    model=finetuned_model,
    tokenizer=tokenizer)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])